<a href="https://colab.research.google.com/github/Jack1447/Medical-Interpretable-LLM/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### MLP在复杂数据集上的分类与决策过程可视化

In [2]:
import torch
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.optim as optim
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
import statistics
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import wandb

建立数据集类

In [3]:
def generate_dataset(batch_size):
    # 下载数据集
    train_transformer=transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=10),
        transforms.ColorJitter(brightness=0.2,contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5],std=[0.5])
    ])

    test_transformer=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5],std=[0.5])
    ])


    train_dataset=datasets.FashionMNIST(
        root='./data',train=True,download=True,transform=train_transformer
    )
    test_dataset=datasets.FashionMNIST(
        root='./data',train=False,download=True,transform=test_transformer
    )

    train_dataloader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True,pin_memory=True)
    test_dataloader=DataLoader(test_dataset,batch_size=batch_size,shuffle=False,pin_memory=True)


    print(f"训练集数目:{len(train_dataset)} 测试集数目:{len(test_dataset)}")
    print(f"训练集批数目:{len(train_dataloader)} 测试集批数目:{len(test_dataloader)}\n")

    return train_dataloader,test_dataloader


建立MLP模型

In [4]:
class ClassMLP(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.layers(x)


指标计算及可视化

In [5]:
def calculate_metrics(label,predicted,average):
    if label.is_cuda:
        label=label.cpu()
    if predicted.is_cuda:
        predicted=predicted.cpu()

    label=label.numpy()
    predicted=predicted.numpy()   # sklearn函数需要numpy数组

    # 准确率
    accuracy=accuracy_score(label,predicted)

    # 精确率 召回率 F1分数
    precision=precision_score(label,predicted,average=average,zero_division=0)
    recall=recall_score(label,predicted,average=average,zero_division=0)
    f1=f1_score(label,predicted,average=average,zero_division=0)

    return accuracy,precision,recall,f1


def validate(model,test_dataloader,criterion,device):
    model.eval()

    with torch.no_grad():
        total_loss=0
        for image,label in test_dataloader:
            image,label=image.to(device),label.to(device)
            output=model(image)
            loss=criterion(output,label)
            total_loss+=loss.item()
    model.train()

    return total_loss/len(test_dataloader)

# 绘制混淆矩阵
def plot_confusion_matrix(model,test_dataloader,device):
    model.eval()
    preds=[]
    labels=[]

    with torch.no_grad():
        for image,label in test_dataloader:
            image,label=image.to(device),label.to(device)
            output=model(image)   # 可见 model 以batch的形式输入
            predicted = torch.argmax(output, dim=1)

            preds.append(predicted.cpu().numpy())
            labels.append(label.cpu().numpy())

    preds=np.concatenate(preds)
    labels=np.concatenate(labels)
    cm = confusion_matrix(labels, preds)

    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("Labels")
    plt.title("Confusion Matrix")
    plt.savefig("Confusion Matrix.png")
    plt.close()

    wandb.log({"Confusion Matrix": wandb.Image("Confusion Matrix.png")})

开始训练

In [6]:
def train(model,train_dataloader,test_dataloader,epochs,device):
    # 定义优化器和学习率调度器
    optimizer = optim.AdamW(
        model.parameters(),
        lr=5e-3,
        weight_decay=0.01
    )

    scheduler = optim.lr_scheduler.CosineAnnealingLR(
        optimizer,
        T_max=epochs,
        eta_min=1e-5
    )

    # 损失函数
    criterion=nn.CrossEntropyLoss()

    # 初始化
    wandb.init(
    project="FashionMNIST",
    name=f"MLP_Class_Model",
    config={
        "epochs": epochs,
        "device": str(device),
        "optimizer": str(optimizer),
        "scheduler": str(scheduler),
        'criterion':str(criterion),
        "model_parameters": sum(p.numel() for p in model.parameters()),
    }
    )

    # 记录模型架构
    wandb.watch(model, log="all", log_freq=10)


    model=model.to(device)
    model.train()
    for epoch in range(epochs):
        total_loss=0
        accuracies=[]
        precisions=[]
        recalls=[]
        f1_scores=[]

        # if (epoch+1)%20==0:
        pbar=tqdm(train_dataloader,desc=f'Train:{epoch+1}/{epochs}',unit='batch')
        # else:
            # pbar=train_dataloader
        for batch_idx,(image,label) in enumerate(pbar):
            image,label=image.to(device),label.to(device)

            optimizer.zero_grad()
            output=model(image)
            loss=criterion(output,label)
            loss.backward()
            optimizer.step()

            total_loss+=loss.item()
            if (batch_idx+1)%50==0:
                pbar.set_postfix({
                    'current loss':loss.item(),
                    'avg loss':total_loss/(batch_idx+1)
                })

            # 计算各项指标
            _,predicted=torch.max(output,dim=1)
            average='weighted'
            accuracy,precision,recall,f1=calculate_metrics(label,predicted,average)
            accuracies.append(accuracy)
            precisions.append(precision)
            recalls.append(recall)
            f1_scores.append(f1)

        # 计算损失
        train_loss=total_loss/len(train_dataloader)
        test_loss=validate(model,test_dataloader,criterion,device)

        # 打印损失
        # if (epoch+1)%20==0:
        print(f"Epoch:{epoch+1} train loss:{train_loss:.4f} test_loss:{test_loss:.4f}")

        accuracy=statistics.mean(accuracies)
        precision=statistics.mean(precisions)
        recall=statistics.mean(recalls)
        f1=statistics.mean(f1_scores)
        print(f"accuracy:{accuracy} precision:{precision} recall:{recall} f1:{f1}\n")

        # 记录损失及指标
        wandb.log({
            'epoch':epoch+1,
            'train loss':train_loss,
            'test loss':test_loss,
            'learning_rate': optimizer.param_groups[0]['lr'],
            'accuracy':accuracy,
            'precision':precision,
            'recall':recall,
            'f1':f1,
        })


        scheduler.step()

    # 生成混淆矩阵 预测可视化
    plot_confusion_matrix(model,test_dataloader,device)

    torch.save(model.state_dict(),'MNIST_Class_Model.pth')
    print(f"模型保存在 'MNIST_Class_Model.pth'")
    print("==================训练完成===================")

    wandb.finish()


主函数

In [7]:
def main():
    batch_size=128
    epochs=35
    train_dataloader,test_dataloader=generate_dataset(batch_size)

    # 实例化模型
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model=ClassMLP(28*28,10)

    print("==================开始训练==================")
    print(f"device:{device} 模型的参数:{sum(p.numel() for p in model.parameters())}\n")
    train(model,train_dataloader,test_dataloader,epochs,device)


if __name__=="__main__":
    main()

100%|██████████| 26.4M/26.4M [00:02<00:00, 11.6MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 209kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.88MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 12.6MB/s]


训练集数目:60000 测试集数目:10000
训练集批数目:469 测试集批数目:79

==================开始训练==================
device:cpu 模型的参数:568970



/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

...

Train:34/35: 100%|██████████| 469/469 [00:59<00:00,  7.94batch/s, current loss=0.18, avg loss=0.227]
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch:34 train loss:0.2273 test_loss:0.2737
accuracy:0.9116304637526652 precision:0.9181748166484776 recall:0.9116304637526652 f1:0.9116699221601545



Train:35/35: 100%|██████████| 469/469 [01:00<00:00,  7.75batch/s, current loss=0.21, avg loss=0.23]
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch:35 train loss:0.2301 test_loss:0.2751
accuracy:0.9127076670220327 precision:0.9190030756559149 recall:0.9127076670220327 f1:0.9127326673580692

模型保存在 'MNIST_Class_Model.pth'
==================训练完成===================


accuracy,▁▃▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇██████████
epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
f1,▁▃▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇██████████
learning_rate,███████▇▇▇▇▆▆▆▆▅▅▅▄▄▄▃▃▃▃▂▂▂▂▁▁▁▁▁▁
precision,▁▃▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇██████████
recall,▁▃▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇██████████
test loss,█▇▇▅▅▅▄▃▄▄▃▃▄▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
train loss,█▆▅▅▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
accuracy,0.91271
epoch,35
f1,0.91273
